In [7]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import asyncio
import pandas as pd

In [8]:
SEASONS = list(range(2019, 2026))

In [9]:
SEASONS

[2019, 2020, 2021, 2022, 2023, 2024, 2025]

In [10]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR)

In [11]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=False)
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        )
        page = await context.new_page()
        
        try:
            for i in range(1, retries + 1):
                if i > 1:
                    await asyncio.sleep(sleep * i)
                
                try:
                    await page.goto(url, timeout=60000)
                    await page.wait_for_selector(selector, timeout=30000)
                    print(await page.title())
                    html = await page.inner_html(selector)
                    break
                    
                except PlaywrightTimeout:
                    print(f"Timeout error on {url} (attempt {i}/{retries})")
                    if i == retries:
                        print(f"Failed after {retries} attempts")
                except Exception as e:
                    print(f"Error on {url}: {e} (attempt {i}/{retries})")
                    if i == retries:
                        print(f"Failed after {retries} attempts")
                        
        finally:
            await browser.close()
    
    return html

In [12]:
async def scrape_season(season):
  url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
  html = await get_html(url, "#content .filter")

  soup = BeautifulSoup(html, 'html.parser')
  links = soup.find_all('a')
  href = [l["href"] for l in links]
  standings_pages = [f"https://basketball-reference.com{l}" for l in href]

  for url in standings_pages:
    save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
    if os.path.exists(save_path):
      continue

    html = await get_html(url, "#all_schedule")
    with open(save_path, "w+") as f:
      f.write(html)

In [13]:
for season in SEASONS:
  await scrape_season(season)

2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2024-25 NBA Schedule | Basketball-Reference.com


In [14]:
standings_files =  os.listdir(STANDINGS_DIR)

In [15]:
async def scrape_game(standings_file):
  with open(standings_file, 'r') as f:
    html = f.read()
  
  soup = BeautifulSoup(html)
  links = soup.find_all("a")
  hrefs = [l.get("href") for l in links]
  box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
  box_scores =  [f"https://www.basketball-reference.com{l}" for l in box_scores]

  for url in box_scores:
    save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
    if os.path.exists(save_path):
      continue

    html = await get_html(url, "#content")
    if not html:
      continue
    with open(save_path, "w+") as f:
      f.write(html)

In [ ]:
for f in standings_files:
  filepath = os.path.join(STANDINGS_DIR, f)

  await scrape_game(filepath) 

In [17]:
await scrape_season(2026)

2025-26 NBA Schedule | Basketball-Reference.com
2025-26 NBA Schedule | Basketball-Reference.com
2025-26 NBA Schedule | Basketball-Reference.com
2025-26 NBA Schedule | Basketball-Reference.com
2025-26 NBA Schedule | Basketball-Reference.com
2025-26 NBA Schedule | Basketball-Reference.com
2025-26 NBA Schedule | Basketball-Reference.com
2025-26 NBA Schedule | Basketball-Reference.com


In [2]:
current_season = 2026

In [5]:
def process_season_schedule(current_season):
  #get the schedule file for the current season
  schedule_files = [f for f in os.listdir(STANDINGS_DIR) if f.startswith(f"NBA_{current_season}")]

  played_games = []
  upcoming_games = []

  for schedule_file in schedule_files:
    filepath = os.path.join(STANDINGS_DIR, schedule_file)

    with open(filepath, 'r') as f:
      html = f.read()
    
    soup = BeautifulSoup(html, 'html.parser')
    
    #find the schedule table
    schedule_table = soup.find('table', {'id': 'schedule'})
    if not schedule_table:
      continue

    rows = schedule_table.find('tbody').find_all('tr')

    for row in rows:
      #skip header rows
      if row.get('class') and 'thead' in row.get('class'):
        continue
      #check if game has been played ( has a box score link)
      box_score_link = row.find('td', {'data-stat': 'box_score_text'})

      if box_score_link and box_score_link.find('a'):
        #game has been played
        played_games.append({
          'date': row.find('th', {'data-stat': 'date_game'}).text if row.find('th', {'data-stat': 'date_game'}) else '',
          'visitor': row.find('td', {'data-stat': 'visitor_team_name'}).text if row.find('td', {'data-stat': 'visitor_team_name'}) else '',
          'home': row.find('td', {'data-stat': 'home_team_name'}).text if row.find('td', {'data-stat': 'home_team_name'}) else '',
          'box_score': box_score_link.find('a')['href'] if box_score_link.find('a') else ''
        })
      else:
        #game is upcoming
        upcoming_games.append({
          'date': row.find('th', {'data-stat': 'date_game'}).text if row.find('th', {'data-stat': 'date_game'}) else '',
          'visitor': row.find('td', {'data-stat': 'visitor_team_name'}).text if row.find('td', {'data-stat': 'visitor_team_name'}) else '',
          'home': row.find('td', {'data-stat': 'home_team_name'}).text if row.find('td', {'data-stat': 'home_team_name'}) else ''
        })
  # save to csv file
  if played_games:
    played_df = pd.DataFrame(played_games)
    played_df.to_csv(os.path.join(DATA_DIR, f'played_games_{current_season}.csv'), index=False)
    print(f"Saved {len(played_games)} played games")

  if upcoming_games:
        upcoming_df = pd.DataFrame(upcoming_games)
        upcoming_df.to_csv(os.path.join(DATA_DIR, f'upcoming_games_{current_season}.csv'), index=False)
        print(f"Saved {len(upcoming_games)} upcoming games")
  return played_games, upcoming_games


In [19]:
# RE-RUN TO UPDATE FOR PREDICTION
played, upcoming = process_season_schedule(2026)

Saved 596 played games
Saved 635 upcoming games


In [21]:
def update_completed_games(predictions_file, played_games_file, season=2026):
  # load predictions
  if not os.path.exists(predictions_file):
    return None
  
  predictions = pd.read_csv(predictions_file)

  #load played games
  played = pd.read_csv(played_games_file)

  #normalize dates for matching
  predictions['date'] = pd.to_datetime(predictions['date']).dt.strftime('%Y-%m-%d')
  played['date'] = pd.to_datetime(played['date'], format='mixed').dt.strftime('%Y-%m-%d')

  #initialize result column if not exists
  if 'result' not in predictions.columns:
    predictions['result'] = 'not_played'
  if 'actual_winner' not in predictions.columns:
    predictions['actual_winner'] = None
  
  #check each prediction against played games
  for idx, pred in predictions.iterrows():
    #find matching game in played games
    match = played[
      (played['date'] == pred['date']) &
      (played['home'] == pred['home']) &
      (played['visitor'] == pred['visitor'])
    ]

    if len(match) > 0:
      # game has been played
      game = match.iloc[0]
      
      # compare data
      if 'winner' in played.columns:
        actual_winner = game['winner']
        predictions.loc[idx, 'actual_winner'] = actual_winner

        if pred['predicted_winner'] == actual_winner:
          predictions.loc[idx, 'result'] = 'correct'
        else:
          predictions.loc[idx, 'result'] = 'incorrect'
    else:
      predictions.loc[idx, 'result'] = 'played_needs_winner'
  
  #save updated predictions
  predictions.to_csv(predictions_file, index=False)
  

  # Print summary
  print("=" * 50)
  print("PREDICTION RESULTS SUMMARY")
  print("=" * 50)
  total = len(predictions)
  correct = len(predictions[predictions['result'] == 'correct'])
  incorrect = len(predictions[predictions['result'] == 'incorrect'])
  not_played = len(predictions[predictions['result'] == 'not_played'])
  needs_winner = len(predictions[predictions['result'] == 'played_needs_winner'])
    
  print(f"Total predictions: {total}")
  print(f"Correct: {correct}")
  print(f"Incorrect: {incorrect}")
  print(f"Not played yet: {not_played}")
  if needs_winner > 0:
    print(f"Played (needs winner lookup): {needs_winner}")
    
  if correct + incorrect > 0:
    accuracy = correct / (correct + incorrect) * 100
    print(f"\nAccuracy: {accuracy:.2f}%")
  print("=" * 50)
    
  return predictions

In [22]:
TEAM_MAP = {
    'Atlanta Hawks': 'ATL', 'Boston Celtics': 'BOS', 'Brooklyn Nets': 'BRK',
    'Charlotte Hornets': 'CHO', 'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL', 'Denver Nuggets': 'DEN', 'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC', 'Los Angeles Lakers': 'LAL', 'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA', 'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS'
  }
  

In [ ]:
def generate_predictions(model, df, predictors, upcoming_file, team=None, n_games=3):
  
  #load upcoming games
  upcoming = pd.read_csv(upcoming_file)
  upcoming['date'] = pd.to_datetime(upcoming['date'], format='mixed')

  #map team names to abbreviations
  upcoming['home_abbrev'] = upcoming['home'].map(TEAM_MAP)
  upcoming['visitor_abbrev'] = upcoming['visitor'].map(TEAM_MAP)

  # filter for specific team if requested
  if team:
    upcoming = upcoming[
      (upcoming['home_abbrev'] == team) |
      (upcoming['visitor_abbrev'] == team)
    ]

  #sort by date and get next n_games per team